# 搭建 Triton 推理服务

本节介绍如何下载、安装、配置、启动 Triton 服务，以及如何用客户端获取 Triton 的推理结果。

GitHub: [triton-inference-server/server](https://github.com/triton-inference-server/server)

## 1. 安装 Nvidia 预构建的 Docker 镜像

Docker Tags: [tritonserver/tags](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver/tags)

从上述链接中，选择适合自己本地环境的 Triton 镜像。我选的是一个比较新的版本。

```
docker pull nvcr.io/nvidia/tritonserver:24.10-py3
```

## 2. 准备模型存储库

### 1）模型存储库结构

模型存储库结构如下：

```
model_repository/
  └── mlp_model/
      ├── 1/
      │   └── model.onnx
      └── config.pbtxt
```

说明：

- `1/`：这是模型的版本号目录，Triton 允许同一个模型存放不同的版本，这里用 1 表示第一个版本。每个版本目录下放置相应的模型文件
- `model.onnx`：每个目录下的 ONNX 模型文件都命名为 model.onnx。你可以将你的模型重命名为 model.onnx

### 2）创建配置文件

每个模型需要一个配置文件 `config.pbtxt`，用于描述模型的输入、输出和其他配置信息。

**MLP 模型的配置**：mlp_model/config.pbtxt

```
name: "mlp_model"
backend: "onnxruntime"
max_batch_size: 64
input [
  {
    name: "x"
    data_type: TYPE_FP32
    dims: [ 7 ]
  }
]
output [
  {
    name: "predict"
    data_type: TYPE_FP32
    dims: [ 2 ]
  }
]
instance_group [
  {
    count: 1
    kind: KIND_GPU
  }
]
dynamic_batching {
  max_queue_delay_microseconds: 100
}
```

> **Note:** 动态批处理
>
> Triton Inference Server 支持动态批处理，即在短时间内将多个请求合并成一个批次进行处理，以最大化 GPU 的利用率。即使将 `max_batch_size` 设置得较大，Triton 也会根据请求的到达情况动态地合并请求，这对于提高吞吐量非常有用。
> 
> 如需使用动态批处理，可在 `config.pbtxt` 中添加如下配置：
>
> ```bash
> dynamic_batching {
  preferred_batch_size: [ 8, 16, 32 ]
  max_queue_delay_microseconds: 100
}
> ```

PS: 进入容器后，使用 `cat /opt/tritonserver/TRITON_VERSION` 命令，可查看镜像的 Triton 版本。

## 3. 启动 Triton 服务

使用以下命令启动 Triton 服务：

```
docker run \
    --rm \
    --gpus '"device=0"' \
    --shm-size=1g \
    -p 8000:8000 \
    -p 8001:8001 \
    -p 8002:8002 \
    -v /path/to/model_repository:/models \
    -v /path/to/workspace:/workspace \
    -v /path/to/data:/workspace/data \
    nvcr.io/nvidia/tritonserver:24.10-py3 \
    tritonserver \
        --model-repository=/models \
        --model-control-mode=poll \
        --repository-poll-secs=30
```

说明：

- `--gpus '"device=0"'`：使用 0 号 GPU
- `--rm`：在容器停止运行后自动删除容器
- `-p 8000:8000、-p 8001:8001、-p 8002:8002`：映射 Triton 的 HTTP、gRPC 和 Prometheus 端口
- `-v /path/to/model_repository:/models`：将本地模型仓库挂载到容器的 /models 目录
- `tritonserver --model-repository=/models`：启动 Triton Server 并指定模型存储库的位置
- `--model-control-mode=poll`：设置模型控制模式为“轮询”。这意味着当模型存储库更新时，Triton 会自动加载
- `--repository-poll-secs=30`：设置 Triton 服务器每 30 秒轮询一次模型仓库，检查是否有模型更新

注意：`/path/to/model_repository` 需要换成你的本地真实目录

## 4. 使用 Triton 推理

In [1]:
# !pip install tritonclient[all]

In [2]:
import requests
import numpy as np
# import json
import tritonclient.http as httpclient

In [3]:
TRITON_URL = "http://localhost:8000"

MLP_MODEL_NAME = 'mlp_model'

### 1) 检查 Triton 的状态

Windows 用 WSL 安装 Ubuntu：

```bash
# 查看可用的发行版
wsl --list --online

# 安装 Ubuntu
wsl --install -d Ubuntu

# 切换到 Ubuntu 并安装 curl
sudo apt update
sudo apt install curl -y

# 下一次进入 Ubuntu 环境
wsl -d Ubuntu
```

现在可以用 curl 测试 Triton 服务:

```
curl -v http://localhost:8000/v2
```

或者用 `requests` 库:

In [4]:
def check_triton_health(triton_url=TRITON_URL):
    url = f"{triton_url}/v2/health/ready"
    response = requests.get(url)
    
    if response.status_code == 200:
        print("Triton server is ready.")
    else:
        print(f"Triton server is not ready. Status code: {response.status_code}")

check_triton_health()

Triton server is ready.


In [5]:
def check_model_health(model_name, triton_url=TRITON_URL):
    url = f"{triton_url}/v2/models/{model_name}/ready"
    response = requests.get(url)
    
    if response.status_code == 200:
        print(f"Model '{model_name}' is ready.")
    else:
        print(f"Model '{model_name}' is not ready. Status code: {response.status_code}")

check_model_health(model_name=MLP_MODEL_NAME)

Model 'mlp_model' is ready.


In [6]:
def get_triton_meta_data(triton_url=TRITON_URL):
    url = f"{triton_url}/v2"
    response = requests.get(url)
    
    if response.status_code == 200:
        metadata = response.json()
        return metadata
    else:
        return ''

get_triton_meta_data()

{'name': 'triton',
 'version': '2.51.0',
 'extensions': ['classification',
  'sequence',
  'model_repository',
  'model_repository(unload_dependents)',
  'schedule_policy',
  'model_configuration',
  'system_shared_memory',
  'cuda_shared_memory',
  'binary_tensor_data',
  'parameters',
  'statistics',
  'trace',
  'logging']}

## 2）HTTP/REST 客户端

可以用 REST API 构造请求。

In [12]:
def client(input_data,
           input_name,
           model_name,
           datatype='FP32',
           triton_url=TRITON_URL,
           model_version='1'):
    url = f"{triton_url}/v2/models/{model_name}/versions/{model_version}/infer"

    # 构造请求体
    inputs = {
        "inputs": [
            {
                "name": input_name,
                "shape": list(input_data.shape),
                "datatype": datatype,
                "data": input_data.flatten().tolist()
            }
        ]
    }

    # 发送 POST 请求
    response = requests.post(url=url, json=inputs)
    if response.status_code != 200:
        raise Exception(f"Inference request failed with status code {response.status_code}: {response.text}")

    return response.json()

In [13]:
# 构造输入
input_data = np.array([np.random.rand(7).astype(np.float32)])

# 发送推理请求
output = client(input_data=input_data,
                input_name="x",
                model_name=MLP_MODEL_NAME)
output

{'model_name': 'mlp_model',
 'model_version': '1',
 'outputs': [{'name': 'predict',
   'datatype': 'FP32',
   'shape': [1, 2],
   'data': [0.21256819367408752, -0.06383021175861359]}]}

### 3） httpclient 客户端

In [9]:
# 创建客户端
client = httpclient.InferenceServerClient("localhost:8000")

# help(client.infer)

In [10]:
# 构造输入
input_data = np.array([np.random.rand(7).astype(np.float32)])

# 请求推理
inputs = [httpclient.InferInput("x", input_data.shape, "FP32")]
inputs[0].set_data_from_numpy(input_data)

outputs = [httpclient.InferRequestedOutput("predict")]

response = client.infer(MLP_MODEL_NAME,
                        inputs=inputs,
                        model_version='1',
                        outputs=outputs)
text_features = response.as_numpy("predict")

text_features.shape

(1, 2)

In [11]:
text_features

array([[ 0.25953004, -0.10462983]], dtype=float32)

参考：

- [triton-inference-server/tutorials](https://github.com/triton-inference-server/tutorials)
- [model_configuration](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md)